In [2]:
!pip install sentence-transformers
!pip install transformers
!pip install faiss-cpu


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.3/245.3 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 55.5 MB/s eta 0:00:00


In [6]:
from sentence_transformers import SentenceTransformer
from transformers import pipeline
import faiss
import numpy as np

# Load a pre-trained embedding model and LLM
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')
llm = pipeline('text-generation', model='gpt2')

# Example knowledge base documents
documents = [
    "The capital of France is Paris.",
    "Python is a programming language popular for machine learning.",
    "The sun is the star at the center of the Solar System."
]

# Generate embeddings for the documents
document_embeddings = embedding_model.encode(documents)
document_embeddings = np.array(document_embeddings)

# Create a FAISS index for fast similarity search
index = faiss.IndexFlatL2(document_embeddings.shape[1])
index.add(document_embeddings)

def retrieve(query, top_k=2):
    # Embed the query
    query_embedding = embedding_model.encode([query])

    # Search the index for similar documents
    distances, indices = index.search(query_embedding, top_k)
    return [documents[i] for i in indices[0]]

def generate_response(query):
    # Retrieve relevant documents
    retrieved_docs = retrieve(query)

    # Format the input for the LLM
    context = " ".join(retrieved_docs)
    prompt = f"Context: {context}\n\nQuestion: {query}\nAnswer:"

    # Generate a response using the LLM
    response = llm(prompt, max_length=100, num_return_sequences=1)
    return response[0]['generated_text']

# Example usage
query = "What is the center of Solar System?"
response = generate_response(query)
print(response)


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Context: The sun is the star at the center of the Solar System. The capital of France is Paris.

Question: What is the center of Solar System?
Answer: Its interior is approximately a hundred thousand light years away, or less than seven degrees. [13] This means that its center is surrounded by the sun about 500 million times. To some extent, this means that it is surrounded by Earth's gravitational pull, just as the Earth's gravitational pull influences the sun by creating


In [7]:
query = "What is the center of French?"
response = generate_response(query)
print(response)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Context: The capital of France is Paris. The sun is the star at the center of the Solar System.

Question: What is the center of French?
Answer: The Sun is not a planet but is a giant star that appears in a few hundred years. French astronomers know it as a constellation.

Question: What is the number of years it took the Sun to arrive inside Paris, which is more than six Earth days? After that the astronomers can also measure the diameter of
